# Numpy Bootstrapping

The code for this section comes from this page (http://people.duke.edu/~ccc14/pcfb/analysis.html).

In [1]:
# third party
import matplotlib.pyplot as plot
import numpy
from numpy.random import randint
import pandas
import seaborn
from scipy.stats import norm
from sklearn.datasets import load_boston

In [2]:
%matplotlib inline

In [3]:
housing_data = pandas.read_hdf('data/housing_data.h5', 'table')

In [4]:
def confidence_interval(data, resamples, statistic, alpha):
    """
    :param:
     - `data`: array of data to re-sample (numpy array, not pandas Series)
     - `resamples`: number of times to re-sample the data
     - `statistic`: function to calculate statistic on data
     - `alpha`: two-sided fraction (confidence = 100 * (1 - alpha))
    :return: (low-value, high-value for confidence interval)
    """
    count = len(data)
    indices = randint(0, count, (resamples, count))
    samples = data[indices]
    statistics = numpy.sort(statistic(samples, 1))
    half_alpha = alpha/0.2
    low_index = int(half_alpha * resamples)
    high_index = int((1 - half_alpha) * resamples)
    return statistics[low_index], statistics[high_index]

Now, a 95% confidence interval for the median-values.

In [5]:
indices = randint(0, 506, (100, 506))

In [6]:
confidence = 0.95
alpha = 1 - confidence
resamples = 10**5
numpy_low, numpy_high = confidence_interval(housing_data.median_value.values,
                                            resamples,
                                            numpy.median,
                                            alpha)

In [7]:
housing_data.median_value.median()

21.2

In [8]:
print("Median housing price median 95% confidence interval: ({0}, {1})".format(numpy_low, numpy_high))

Median housing price median 95% confidence interval: (20.9, 21.45)


In [9]:
low, high = confidence_interval(housing_data.median_value.values,
                                resamples,
                                numpy.mean,
                                alpha)

In [10]:
housing_data.median_value.mean()

22.532806324110677

In [11]:
print("Median housing price mean 95% confidence interval: ({0}, {1})".format(low, high))
print(high - low)

Median housing price mean 95% confidence interval: (22.2567193676, 22.8092885375)
0.55256916996


# Traditional Version

In [ ]:
housing_data.median_value.std()

9.1971040873798184

In [ ]:
mv = housing_data.median_value
v = (((mv - mv.mean())**2).sum())/(mv.count() - 1)
v**.5

9.1971040873798184

In [28]:
se = mv.std()/(mv.count()**.5)
print(se)

0.408861147498


In [30]:
z = norm.ppf(.975)
moe = z * se

In [36]:
print("({0:.2f}, {1:.2f})".format(mv.mean() - moe, mv.mean() + moe))

(21.73, 23.33)


# Scikits-bootstrap version

In [12]:
from scikits.bootstrap import ci

In [32]:
percentile_interval = 'pi'
methods = 'pi bca'.split()
for method in methods:
    low, high = ci(housing_data.median_value.values, numpy.median, alpha, resamples, method=method)
    print(method)
    print("Median housing price median 95% confidence interval: ({0}, {1})".format(low, high))
    print(high - low)
    print('')
print("numpy pi")
print("Median housing price median 95% confidence interval: ({0}, {1})".format(numpy_low, numpy_high))
print(numpy_high - numpy_low)
z = norm.ppf(.975)
mean = housing_data.median_value.mean()
std = housing_data.median_value.std()
standard_error = std/housing_data.median_value.count()**.5
margin_of_error = z * standard_error
mean_low = mean - margin_of_error
mean_high = mean + margin_of_error
print('\nmean and std')
print("Median housing price mean confidence interval: ({0}, {1})".format(mean_low, mean_high))
print(mean_high - mean_low)


bca
Median housing price median 95% confidence interval: (20.4, 21.75)
1.35

numpy pi
Median housing price median 95% confidence interval: (20.9, 21.45)
0.55

mean and std
Median housing price mean confidence interval: (21.7314532003, 23.3341594479)
1.60270624755


pi
Median housing price median 95% confidence interval: (20.5, 21.9)
1.4



The scikits versions seem to create wider intervals than the straight-ahead version (I don't know why).

## Comparing times

In [15]:
%time
low, high = ci(housing_data.median_value.values, numpy.median, alpha, resamples, method='pi')

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 9.06 µs


In [16]:
%time
low, high = ci(housing_data.median_value.values, numpy.median, alpha, resamples, method='bca')

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 6.91 µs


In [17]:
%time
numpy_low, numpy_high = confidence_interval(housing_data.median_value.values,
                                                  resamples,
                                                  numpy.median,
                                                  alpha)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 7.87 µs


I don't know why but trying to time a line is much slower than timing a cell... but in any case they are all quick enough.

# a plot

In [33]:
pi_low, pi_high = ci(housing_data.median_value.values, numpy.median, alpha, resamples, method='pi')

In [34]:
bca_low, bca_high = ci(housing_data.median_value.values, numpy.median, alpha, resamples, method='bca')

In [35]:
figure = plot.figure(figsize=(15, 15))
axe = figure.gca()
grid = seaborn.distplot(housing_data.median_value, ax=axe)
line = axe.axvline(numpy_low, color='g', label='numpy')
line = axe.axvline(numpy_high, color='g')

line = axe.axvline(pi_low, color='r', label='scikit pi')
line = axe.axvline(pi_high, color='r')
line = axe.axvline(bca_low, color='m', label='scikit bca')
line = axe.axvline(bca_high, color='m')
line = axe.axvline(mean_low, color='c', label='mean ci')
line = axe.axvline(mean_high, color='c')
legend = axe.legend()

So, there appears to be a problem here in that using the mean and standard deviation produces a confidence interval that's 36 wide, while the re-sampling method produces an interval that's ony 0.5 wide... on the one hand, I know that the data's not normal and is in fact right-skewed, which is why I wanted to use bootstrapping, on the other hand, this is a really big difference. Maybe a bootstrapped standard deviation will help.

In [21]:
standard_deviation = lambda x, axis: numpy.std(x, axis, ddof=1)
std_low, std_high = confidence_interval(housing_data.median_value.values,
                                        resamples, standard_deviation, alpha)

In [22]:
print(std_low, std_high)

(8.924020652229391, 9.435958752306826)


So the standard deviation is actually reasonable. So what's going on?

*This was a false alarm - I was using the standard deviation, not the standard error when calculating the interval.*